In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
import torch.nn.functional as F


import os
import time
import tqdm
import errno
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from timeit import default_timer as timer

import importlib
import utils
import networks

In [8]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(torch.cuda.is_available())
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    
def save_model(path, filename, data):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(data, os.path.join(path, filename))
    pass

def save_loss(path, filename, data):
    if not os.path.exists(path):
        os.makedirs(path)
    data.to_csv(os.path.join(path, filename), index=False)
    pass

cuda


# Hyperparameters and Data Sampling

In [9]:
# model parameters
K = 10
r = 0.035
sigma = 0.2
T = 1
S_range = [0, int(5*K)]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)
M = 100
N = 5000

# model parameters
lossFunction = nn.MSELoss()
sizes=[2, 50, 50, 50, 50, 50, 50, 50, 50, 1]
lr = 3e-5
activation = 'relu'

# training parameters
samples = {"pde": 5000, "bc":500, "fc":500}

# sample data generated by finite difference method
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor = utils.fdm_data(S_range[-1], T, M, N, "500000sample.csv", device)

# Test AWPINN with Different Learning Rate

In [ ]:
# Initialize a list to store the loss histories
n_epochs = 5000
lr_rate_list = [lr/10000, lr/100, lr/10, lr, lr*10, lr*100]

In [ ]:
for i in range(len(lr_rate_list)):
    # Initialize lists to store the loss histories for all components
    weight_lr = lr_rate_list[i]
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    
    path = f"test_learning_rate/awpinn/v2/{weight_lr}/"

    # run 10 times for each learning rate
    for j in range(10):
        awipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weight_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net='pinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=weight_lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=None, adaptive_rate_scaler=10.0, loss_weights=[], adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
            )
        # # Save the model's state dictionary
        # save_model(f'{path}/model', f"{j}.pth", min_model)
        
        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist
        })
        save_loss(f'{path}/loss', f'{j}.csv', loss_df)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        pass

    # Calculate the average losses among all 10 training sessions for each learning rate
    average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
    average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
    average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
    average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

    # Save the average losses as CSV files
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': average_mse_loss,
        'Average_PDE_Loss': average_pde_loss,
        'Average_BC_Loss': average_bc_loss,
        'Average_Data_Loss': average_data_loss
    })
    save_loss(path, 'average_loss.csv', average_loss_df)

In [ ]:
for i in range(len(lr_rate_list)):
    # Initialize lists to store the loss histories for all components
    weight_lr = lr_rate_list[i]
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    
    path = f"test_learning_rate/awipinn/{weight_lr}/"

    # run 10 times for each learning rate
    for j in range(10):
        awipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net='ipinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=weight_lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=[], adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
            )
        # # Save the model's state dictionary
        # save_model(f'{path}/model', f"{j}.pth", min_model)
        
        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist
        })
        save_loss(f'{path}/loss', f'{j}.csv', loss_df)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        pass

    # Calculate the average losses among all 10 training sessions for each learning rate
    average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
    average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
    average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
    average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

    # Save the average losses as CSV files
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': average_mse_loss,
        'Average_PDE_Loss': average_pde_loss,
        'Average_BC_Loss': average_bc_loss,
        'Average_Data_Loss': average_data_loss
    })
    save_loss(path, 'average_loss.csv', average_loss_df)

# PINN and IPINN with Different Loss weights

In [ ]:
n_epochs = 5000
weights = [[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5], [0.7, 0.15, 0.15], [0.15, 0.7, 0.15], [0.15, 0.15, 0.7], [1, 1, 1]]

In [ ]:
# Initialize lists to store the loss histories for all components
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []

# train models with different weights
for i in range(len(weights)):
    w1 = weights[i][0]
    w2 = weights[i][1]
    w3 = weights[i][2]
    path = f"test_loss_weights/pinn"
    wpinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weights_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='pinn', sizes=sizes, activation='relu', learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, 
        lossFunction=lossFunction, dropout_rate=None, adaptive_rate=None, adaptive_rate_scaler=None, loss_weights=weights[i], adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )
    # save_model(f'{path}/model2', f"{w1}-{w2}-{w3}.pth", min_model)  # Save the model's state dictionary
    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    save_loss(f'{path}/loss10', f'{w1}-{w2}-{w3}.csv', loss_df)
    pass

In [ ]:
# Initialize lists to store the loss histories for all components
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []

# train models with different weights
for i in range(len(weights)):
    w1 = weights[i][0]
    w2 = weights[i][1]
    w3 = weights[i][2]
    path = f"test_loss_weights/ipinn"
    wipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weights_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='ipinn', sizes=sizes, activation='relu', learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, 
        lossFunction=lossFunction, dropout_rate=0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=weights[i], adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )
    # save_model(f'{path}/model2', f"{w1}-{w2}-{w3}.pth", min_model)  # Save the model's state dictionary
    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    save_loss(f'{path}/loss10', f'{w1}-{w2}-{w3}.csv', loss_df)
    pass

# Ibpinn and awipinn with different loss weights

In [ ]:
n_epochs = 5000
awpinn_weights = [[1.5, 1, 1], [5, 1, 1], [1, 1, 2], [1, 2, 1]]

In [ ]:
for i in range(len(awpinn_weights)):
    # Initialize lists to store the loss histories for all components
    w1 = awpinn_weights[i][0]
    w2 = awpinn_weights[i][1]
    w3 = awpinn_weights[i][2]
    
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    
    path = f"test_loss_weights/awpinn/{w1}-{w2}-{w3}/"

    # run 10 times for each learning rate
    for j in range(10):
        awipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weight_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net='pinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=0.001, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=None, adaptive_rate_scaler=10.0, loss_weights=awpinn_weights[i], adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
            )
        # # Save the model's state dictionary
        # save_model(f'{path}/model', f"{j}.pth", min_model)
        
        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist
        })
        save_loss(f'{path}/loss', f'{j}.csv', loss_df)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        pass

    # Calculate the average losses among all 10 training sessions for each learning rate
    average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
    average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
    average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
    average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

    # Save the average losses as CSV files
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': average_mse_loss,
        'Average_PDE_Loss': average_pde_loss,
        'Average_BC_Loss': average_bc_loss,
        'Average_Data_Loss': average_data_loss
    })
    save_loss(f'{path}', f'{w1}-{w2}-{w3}.csv', loss_df)

In [ ]:
for i in range(len(awpinn_weights)):
    # Initialize lists to store the loss histories for all components
    w1 = awpinn_weights[i][0]
    w2 = awpinn_weights[i][1]
    w3 = awpinn_weights[i][2]
    
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    
    path = f"test_loss_weights/awipinn/{w1}-{w2}-{w3}/"

    # run 10 times for each learning rate
    for j in range(10):
        awipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weight_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net='ipinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=0.001, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=awpinn_weights[i], adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
            )
        # # Save the model's state dictionary
        # save_model(f'{path}/model', f"{j}.pth", min_model)
        
        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist
        })
        save_loss(f'{path}/loss', f'{j}.csv', loss_df)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        pass

    # Calculate the average losses among all 10 training sessions for each learning rate
    average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
    average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
    average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
    average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

    # Save the average losses as CSV files
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': average_mse_loss,
        'Average_PDE_Loss': average_pde_loss,
        'Average_BC_Loss': average_bc_loss,
        'Average_Data_Loss': average_data_loss
    })
    save_loss(f'{path}', f'{w1}-{w2}-{w3}.csv', loss_df)

# PINN vs IPINN vs AWPINN

In [4]:
n_epochs = 10000
# ipinn_loss_weight = [[1, 1, 1], [1, 1.5, 1.5], [1, 2, 2] [1.5, 1, 1], [2, 1, 1]]

In [6]:
def train_network(net_type, path_prefix, loss_weights, adaptive_rate=None, aw_learning_rate=0.001, adaptive_weight=None):
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    all_relative_error_hist = []

    for i in range(10):
        path = f"{path_prefix}/{i}"
        model, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weight_hist, relative_L2_loss_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net=net_type, sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=aw_learning_rate, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=adaptive_rate, adaptive_rate_scaler=10.0 if adaptive_rate else 0, loss_weights=loss_weights, adaptive_weight=adaptive_weight, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor, X_test_tensor=X_test_tensor, y_test_tensor=y_test_tensor
        )

        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist,
            'Relative_L2_Loss': relative_L2_loss_hist
        })
        save_loss(f'{path}/loss', f'{i}.csv', loss_df)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        all_relative_error_hist.append(relative_L2_loss_hist)

    # Calculate and save average losses
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': pd.DataFrame(all_mse_loss_hist).mean(axis=0),
        'Average_PDE_Loss': pd.DataFrame(all_pde_loss_hist).mean(axis=0),
        'Average_BC_Loss': pd.DataFrame(all_bc_loss_hist).mean(axis=0),
        'Average_Data_Loss': pd.DataFrame(all_data_loss_hist).mean(axis=0),
        'Average_Relative_L2_Loss': pd.DataFrame(all_relative_error_hist).mean(axis=0)
    })
    save_loss(path_prefix, 'average_loss.csv', average_loss_df)

In [ ]:
train_network('pinn', 'test_final_performance/pinn/1-1-1', [1,1,1])
train_network('ipinn', 'test_final_performance/ipinn/1-1-1', [1,1,1], adaptive_rate=0.1)
train_network('pinn', 'test_final_performance/awpinn/1-1-1', [1,1,1], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_final_performance/awipinn/1-1-1', [1,1,1], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

In [ ]:
train_network('pinn', 'test_final_performance/pinn/1.5-1-1', [1.5,1,1])
train_network('ipinn', 'test_final_performance/ipinn/1.5-1-1', [1.5,1,1], adaptive_rate=0.1)
train_network('pinn', 'test_final_performance/awpinn/1.5-1-1', [1.5,1,1], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_final_performance/awipinn/1.5-1-1', [1.5,1,1], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

In [ ]:
train_network('pinn', 'test_final_performance/pinn/2-1-1', [2,1,1])
train_network('ipinn', 'test_final_performance/ipinn/2-1-1', [2,1,1], adaptive_rate=0.1)
train_network('pinn', 'test_final_performance/awpinn/2-1-1', [2,1,1], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_final_performance/awipinn/2-1-1', [2,1,1], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

In [ ]:
train_network('pinn', 'test_final_performance/pinn/1-1.5-1.5', [1,1.5,1.5])
train_network('ipinn', 'test_final_performance/ipinn/1-1.5-1.5', [1,1.5,1.5], adaptive_rate=0.1)
train_network('pinn', 'test_final_performance/awpinn/1-1.5-1.5', [1,1.5,1.5], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_final_performance/awipinn/1-1.5-1.5', [1,1.5,1.5], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

In [8]:
train_network('pinn', 'test_final_performance/pinn/1-2-2', [1,2,2])
train_network('ipinn', 'test_final_performance/ipinn/1-2-2', [1,2,2], adaptive_rate=0.1)
train_network('pinn', 'test_final_performance/awpinn/1-2-2', [1,2,2], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_final_performance/awipinn/1-2-2', [1,2,2], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

[Training procedure]: 100%|##########| 10000/10000 [04:00<00:00, 41.61it/s]


# change hidden layers

In [9]:
sizes=[2, 25, 25, 25, 25, 25, 25, 25, 25, 1]

def train_network(net_type, path_prefix, loss_weights, adaptive_rate=None, aw_learning_rate=0.001, adaptive_weight=None):
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    all_relative_error_hist = []

    for i in range(10):
        path = f"{path_prefix}/{i}"
        model, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weight_hist, relative_L2_loss_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net=net_type, sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=aw_learning_rate, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=adaptive_rate, adaptive_rate_scaler=10.0 if adaptive_rate else 0, loss_weights=loss_weights, adaptive_weight=adaptive_weight, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor, X_test_tensor=X_test_tensor, y_test_tensor=y_test_tensor
        )

        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist,
            'Relative_L2_Loss': relative_L2_loss_hist
        })
        save_loss(f'{path}/loss', f'{i}.csv', loss_df)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        all_relative_error_hist.append(relative_L2_loss_hist)

    # Calculate and save average losses
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': pd.DataFrame(all_mse_loss_hist).mean(axis=0),
        'Average_PDE_Loss': pd.DataFrame(all_pde_loss_hist).mean(axis=0),
        'Average_BC_Loss': pd.DataFrame(all_bc_loss_hist).mean(axis=0),
        'Average_Data_Loss': pd.DataFrame(all_data_loss_hist).mean(axis=0),
        'Average_Relative_L2_Loss': pd.DataFrame(all_relative_error_hist).mean(axis=0)
    })
    save_loss(path_prefix, 'average_loss.csv', average_loss_df)
    
train_network('pinn', 'test_diff_nn/25neurons/pinn/1-1-1', [1,1,1])
train_network('ipinn', 'test_diff_nn/25neurons/ipinn/1-1-1', [1,1,1], adaptive_rate=0.1)
train_network('pinn', 'test_diff_nn/25neurons/awpinn/1-1-1', [1,1,1], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_diff_nn/25neurons/awipinn/1-1-1', [1,1,1], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

train_network('pinn', 'test_diff_nn/25neurons/pinn/2-1-1', [2,1,1])
train_network('ipinn', 'test_diff_nn/25neurons/ipinn/2-1-1', [2,1,1], adaptive_rate=0.1)
train_network('pinn', 'test_diff_nn/25neurons/awpinn/2-1-1', [2,1,1], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_diff_nn/25neurons/awipinn/2-1-1', [2,1,1], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

[Training procedure]: 100%|##########| 10000/10000 [02:09<00:00, 77.34it/s]


In [ ]:
sizes=[2, 50, 50, 50, 50, 1]

def train_network(net_type, path_prefix, loss_weights, adaptive_rate=None, aw_learning_rate=0.001, adaptive_weight=None):
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    all_relative_error_hist = []

    for i in range(10):
        path = f"{path_prefix}/{i}"
        model, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weight_hist, relative_L2_loss_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net=net_type, sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=aw_learning_rate, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=adaptive_rate, adaptive_rate_scaler=10.0 if adaptive_rate else 0, loss_weights=loss_weights, adaptive_weight=adaptive_weight, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor, X_test_tensor=X_test_tensor, y_test_tensor=y_test_tensor
        )

        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist,
            'Relative_L2_Loss': relative_L2_loss_hist
        })
        save_loss(f'{path}/loss', f'{i}.csv', loss_df)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        all_relative_error_hist.append(relative_L2_loss_hist)

    # Calculate and save average losses
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': pd.DataFrame(all_mse_loss_hist).mean(axis=0),
        'Average_PDE_Loss': pd.DataFrame(all_pde_loss_hist).mean(axis=0),
        'Average_BC_Loss': pd.DataFrame(all_bc_loss_hist).mean(axis=0),
        'Average_Data_Loss': pd.DataFrame(all_data_loss_hist).mean(axis=0),
        'Average_Relative_L2_Loss': pd.DataFrame(all_relative_error_hist).mean(axis=0)
    })
    save_loss(path_prefix, 'average_loss.csv', average_loss_df)

train_network('pinn', 'test_diff_nn/4_layers/pinn/1-1-1', [1,1,1])
train_network('ipinn', 'test_diff_nn/4_layers/ipinn/1-1-1', [1,1,1], adaptive_rate=0.1)
train_network('pinn', 'test_diff_nn/4_layers/awpinn/1-1-1', [1,1,1], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_diff_nn/4_layers/awipinn/1-1-1', [1,1,1], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

train_network('pinn', 'test_diff_nn/4_layers/pinn/2-1-1', [2,1,1])
train_network('ipinn', 'test_diff_nn/4_layers/ipinn/2-1-1', [2,1,1], adaptive_rate=0.1)
train_network('pinn', 'test_diff_nn/4_layers/awpinn/2-1-1', [2,1,1], aw_learning_rate=0.001, adaptive_weight=True)
train_network('ipinn', 'test_diff_nn/4_layers/awipinn/2-1-1', [2,1,1], aw_learning_rate=0.001, adaptive_rate=0.1, adaptive_weight=True)

In [11]:
import datetime
n_epochs = 10000
net = networks.FeedforwardNeuralNetwork(2, 50, 1, 8)
net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-5)
loss_hist6 = []
loss_hist7 = []
logging.info(f'{net}\n')
logging.info(f'Training started at {datetime.datetime.now()}\n')
min_loss = float("inf")  # Initialize with a large value
final_model = None
start_time = timer()
for _ in tqdm.tqdm(range(n_epochs), desc='[Training procedure]', ascii=True, total=n_epochs):
    prediction = net(X_train_tensor)
    loss = lossFunction(prediction, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_hist6.append(loss.item())
    
    # Calculate the relative L2 loss
    prediction2 = net(X_test_tensor)
    relative_L2 = torch.sqrt(torch.mean((prediction2 - y_test_tensor)**2)) / torch.sqrt(torch.mean(y_test_tensor**2))
    loss_hist7.append(relative_L2.item())
    
    if relative_L2.item() < min_loss:
        min_loss = relative_L2.item()
        final_model = net.state_dict()
    pass

# torch.save(final_model, "default/nn.pth")  # Save the model's state dictionary
# # Save the training loss history as a CSV file
# loss_df = pd.DataFrame(loss_hist7)
# loss_df.to_csv('default/nn_loss.csv', index=False)

for i in range(n_epochs):
    print('training', loss_hist6[i], 'relative L2', loss_hist7[i])

[Training procedure]: 100%|##########| 10000/10000 [01:53<00:00, 88.47it/s]

training 422.00335693359375 relative L2 0.9973743557929993
training 421.9865417480469 relative L2 0.9973543882369995
training 421.9696350097656 relative L2 0.997334361076355
training 421.9526672363281 relative L2 0.9973140954971313
training 421.9355773925781 relative L2 0.9972938895225525
training 421.9184265136719 relative L2 0.9972734451293945
training 421.9011535644531 relative L2 0.997252881526947
training 421.8837585449219 relative L2 0.9972321391105652
training 421.8661804199219 relative L2 0.997211217880249
training 421.8485107421875 relative L2 0.9971901774406433
training 421.8307189941406 relative L2 0.9971688985824585
training 421.812744140625 relative L2 0.9971476197242737
training 421.794677734375 relative L2 0.9971259236335754
training 421.7763977050781 relative L2 0.9971041679382324
training 421.7579040527344 relative L2 0.9970819354057312
training 421.73919677734375 relative L2 0.9970595240592957
training 421.72021484375 relative L2 0.997036874294281
training 421.7010192